In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import stockdata as sd
import pandas as pd
import requests
import os
import io
import arrow
import json
import glob
from zipfile import ZipFile

#### *URLs of NSE Data*

***In Progress***
- Daily Volatality File F&O - https://archives.nseindia.com/archives/nsccl/volt/FOVOLT_04052021.csv
- Daily Volatality File Cash Market- https://archives.nseindia.com/archives/nsccl/volt/CMVOLT_07052021.CSV
- 52W HL Report - https://archives.nseindia.com/content/CM_52_wk_High_low_06052021.csv
- Full Bhavcopy & Security Deliverable Data https://archives.nseindia.com/products/content/sec_bhavdata_full_07052021.csv
- Bulk Deals - https://archives.nseindia.com/content/equities/bulk.csv
- Block Deals - https://archives.nseindia.com/content/equities/block.csv
- Indices Data - https://archives.nseindia.com/content/indices/ind_close_all_07052021.csv
- Nifty Top 10 - https://archives.nseindia.com/content/indices/top10nifty50_070521.csv

***Downloaded (as Excel)***
- Market Activity Report (Index Futures OI) - https://archives.nseindia.com/archives/fo/mkt/fo04052021.zip
- FII Derivative Stats - https://archives.nseindia.com/content/fo/fii_stats_04-May-2021.xls
- Participant Wise Trading Volumes - https://archives.nseindia.com/content/nsccl/fao_participant_vol_04052021.csv
- Participant Wise Open Interest - https://archives.nseindia.com/content/nsccl/fao_participant_oi_04052021.csv

***Downloaded (as CSV)***
- NSE Open Interest (Stock Futures OI) - https://archives.nseindia.com/archives/nsccl/mwpl/nseoi_04052021.zip
- F&O Bhavcopy (Futures & Options Snapshot) - https://archives.nseindia.com/content/historical/DERIVATIVES/2021/MAY/fo04MAY2021bhav.csv.zip

### Functions to download different data from NSE for a given date

In [75]:
def getParticipantWiseOI(date):
    url = f'https://archives.nseindia.com/content/nsccl/fao_participant_oi_{date.format("DDMMYYYY")}.csv'
    try:
        df = pd.read_csv(url,  skiprows=1)
    except:
        return pd.DataFrame()
    df.insert(loc = 0, column = 'Date', value = date.format("YYYY-MM-DD"))
    return df


def getParticipantWiseVol(date):
    url = f'https://archives.nseindia.com/content/nsccl/fao_participant_vol_{date.format("DDMMYYYY")}.csv'
    try:
        df = pd.read_csv(url,  skiprows=1)
    except:
        return pd.DataFrame()
    df.insert(loc = 0, column = 'Date', value = date.format("YYYY-MM-DD"))
    return df


def getFIIStats(date):
    url = f'https://archives.nseindia.com/content/fo/fii_stats_{date.format("DD-MMM-YYYY")}.xls'
    try:
        df = pd.read_excel(url)
    except:
        return pd.DataFrame()
    df = df.iloc[2:6]
    df.columns = ['Type', 'Contracts Bought', 'Buy Amt in Crores', 'Contracts Sold', 'Sell Amt in Crores', 'Eod OI', 'Eod OI Amt in Crores']
    df.insert(loc = 0, column = 'Date', value = date.format("YYYY-MM-DD"))
    return df


def getIndexFuturesOI(date):
    try:
        url = f'https://archives.nseindia.com/archives/fo/mkt/fo{date.format("DDMMYYYY")}.zip'
        r = requests.get(url)
        if r.status_code == 404: return pd.DataFrame()
        z = ZipFile(io.BytesIO(r.content))
        cols = ['Symbol', 'Total Contracts', 'Traded Quantity', 'Traded Value In Crs', 'OI']
        df = pd.read_csv(z.open(f'futidx{date.format("DDMMYYYY")}.csv'), skiprows = 2, names = cols)
        df.insert(loc = 0, column = 'Date', value = date.format("YYYY-MM-DD"))
    except:
        return pd.DataFrame()
    return df


def getFNOSymbolsOI(date):
    try:
        url = f'https://archives.nseindia.com/archives/nsccl/mwpl/nseoi_{date.format("DDMMYYYY")}.zip'
        r = requests.get(url)
        if r.status_code == 404: return pd.DataFrame()
        z = ZipFile(io.BytesIO(r.content))
        df = pd.read_csv(z.open(f'nseoi_{date.format("DDMMYYYY")}.csv'))
        df.columns = ['Date', 'ISIN', 'Name', 'Symbol', 'MWPL', 'OI']
        df.Date = df.Date.apply(lambda x: arrow.get(x, 'DD-MMM-YYYY').format('YYYY-MM-DD'))
    except:
        return pd.DataFrame()
    return df


def getDerivativeData(date):
    try:
        cols = ['Instrument', 'Symbol', 'ExpiryDt', 'StrikePrice', 'OptionTp', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng', 'Date']
        new_cols = ['Date', 'Symbol', 'ExpiryDt', 'StrikePrice', 'OptionTp', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng']
        url = f'https://archives.nseindia.com/content/historical/DERIVATIVES/{date.format("YYYY")}/{date.format("MMM").upper()}/fo{date.format("DDMMMYYYY").upper()}bhav.csv.zip'
        s = requests.get(url, timeout=1)
        df = pd.read_csv(url)
        df.drop(df.columns[-1], axis=1, inplace=True)
        df.columns  = cols
        df = df[df.Symbol.isin(['NIFTY', 'BANKNIFTY'])].reset_index()
        df = df[new_cols]
    except:
        print(f'Timeout - {url}')
        return pd.DataFrame()
    return df


def getFIIDIICMActivity(date):
    try:
        url = f'https://www.traderscockpit.com/?pageView=fii-dii-in-india&tradeDate={date.format("MM/DD/YYYY")}'
        x = pd.read_html(url)
        df = pd.concat([x[2], x[3]], ignore_index=True)
        df.insert(loc = 0, column = 'Date', value = date.format("YYYY-MM-DD"))
        df.insert(loc = 1, column = 'Entity', value = 'FII')
        df.loc[1,'Entity'] = 'DII'
    except:
        return pd.DataFrame()
    return df


def getAllData(func, since, till):
    dt = arrow.get(since)
    tilldt = arrow.get(till)
    dfs = []
    while dt <= tilldt:
        if dt.format('d') in ['6', '7']: 
            dt = dt.shift(days=+1)
            continue
        print(dt.format("YYYY-MMM-DD"))
        df = func(dt)
        if not df.empty: dfs.append(df)
        dt = dt.shift(days=+1)
    df = pd.concat(dfs, ignore_index=True)
    return df

### Download Daily Data

*1. Download Participant Wise OI & Vol, FII Stats & Index Futures OI*

In [51]:
%%time
def getDailyEODData(date):
    df1 = getParticipantWiseOI(date)
    df2 = getParticipantWiseVol(date)
    df3 = getFIIStats(date)
    df4 = getIndexFuturesOI(date)
    excel_file = f'/Users/ajit/Downloads/NSE Data/EOD Data - {date.format("YYYY-MM-DD")}.xlsx'
    with pd.ExcelWriter(excel_file) as writer:  
        df1.to_excel(writer, index = False, sheet_name='Participant Wise OI')
        df2.to_excel(writer, index = False, sheet_name='Participant Wise Vol')
        df3.to_excel(writer, index = False, sheet_name='FII Stats')
        df4.to_excel(writer, index = False, sheet_name='Index Futures OI')

getDailyEODData(arrow.get('2021-05-03'))

CPU times: user 147 ms, sys: 36.2 ms, total: 183 ms
Wall time: 3.1 s


2. Download OI of FNO Symbols

In [80]:
date = arrow.get('2021-05-03')
df = getFNOSymbolsOI(date)
df.to_csv(f'/Users/ajit/Downloads/NSE Data/EOD FNO Symbols OI/eod_fno_symbols_oi_{date.format("YYYYMMDD")}.csv', index=False)

3. Download Index Futures & Options EOD Snapshot

In [81]:
def getEODIndexFNOSnapshot(date):
    df = getDerivativeData(date)
    futcols = ['Date', 'Symbol', 'ExpiryDt', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng']
    df1 = df[(df.Symbol.isin(['NIFTY', 'BANKNIFTY'])) & (df.OptionTp == 'XX')]
    df1 = df1[futcols]
    df1.to_csv(f'/Users/ajit/Downloads/NSE Data/nifty_bnf_daily_fut_snapshot_{date.format("YYYYMMDD")}.csv', index=False)
    optcols = ['Date', 'Symbol', 'ExpiryDt', 'StrikePrice', 'OptionTp', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng']
    df2 = df[(df.Symbol == 'NIFTY') & (df.OptionTp != 'XX')]
    df2 = df2[optcols]
    df2.to_csv(f'/Users/ajit/Downloads/NSE Data/EOD Options Snapshot - Nifty/nifty_daily_options_snapshot_{date.format("YYYYMMDD")}.csv', index=False)
    df3 = df[(df.Symbol == 'BANKNIFTY') & (df.OptionTp != 'XX')]
    df3 = df3[optcols]
    df3.to_csv(f'/Users/ajit/Downloads/NSE Data/EOD Options Snapshot - BankNifty/banknifty_daily_options_snapshot_{date.format("YYYYMMDD")}.csv', index=False)
    
getEODIndexFNOSnapshot(arrow.get('2021-05-04'))
getEODIndexFNOSnapshot(arrow.get('2021-05-05'))
getEODIndexFNOSnapshot(arrow.get('2021-05-06'))
getEODIndexFNOSnapshot(arrow.get('2021-05-07'))

### Download Historical Data

In [ ]:
%%time
# df = getAllData(getParticipantWiseOI, '2020-01-01', '2021-05-01')
# df = getAllData(getParticipantWiseVol', 2020-01-01', '2021-05-01')
# df = getAllData(getFIIStats, '2021-01-01', '2021-05-01')
# df = getAllData(getIndexFuturesOI, '2016-01-01', '2018-12-31')
# df = getAllData(getFIIDIICMActivity, '2018-01-01', '2021-05-01')

In [ ]:
df.shape
df = df[~df['Buy Value(In Crs)'].str.contains('data not found', na=False)]
df.shape
df[df.Entity == 'FII'].head(3)
df[df.Entity == 'FII'].tail(3)
df[df.Entity == 'DII'].head(3)
df[df.Entity == 'DII'].tail(3)

In [45]:
df[df.Entity == 'FII'].to_csv('/Users/ajit/Downloads/fii_activity.csv', index=False)
df[df.Entity == 'DII'].to_csv('/Users/ajit/Downloads/dii_activity.csv', index=False)

### Download Futures OI of FNO Symbols

In [ ]:
%%time
# df = getAllData(getFNOSymbolsOI, '2017-01-01', '2020-12-31')

### Download Futures & Options Snapshot of Nifty & BankNifty

In [ ]:
%%time
# df = getAllData(getDerivativeData, '2021-01-01', '2021-05-01')

##### *Combine Nifty and BNF EOD Futures Snapshot data*

In [ ]:
year = '2021'

futcols = ['Date', 'Symbol', 'ExpiryDt', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng']
df1 = df[(df.Symbol.isin(['NIFTY', 'BANKNIFTY'])) & (df.OptionTp == 'XX')]
df1 = df1[futcols]
df1.to_csv(f'/Users/ajit/Downloads/nifty_bnf_daily_fut_snapshot_{year}.csv', index=False)


optcols = ['Date', 'Symbol', 'ExpiryDt', 'StrikePrice', 'OptionTp', 'Open', 'High', 'Low', 'Close', 'SettlePrice', 'Contracts', 'ValueInLakhs', 'OI', 'OIChng']

df2 = df[(df.Symbol == 'NIFTY') & (df.OptionTp != 'XX')]
df2 = df2[optcols]
df2.to_csv(f'/Users/ajit/Downloads/nifty_daily_options_snapshot_{year}.csv', index=False)

df3 = df[(df.Symbol == 'BANKNIFTY') & (df.OptionTp != 'XX')]
df3 = df3[optcols]
df3.to_csv(f'/Users/ajit/Downloads/banknifty_daily_options_snapshot_{year}.csv', index=False)

In [ ]:
df.shape
df.head(3)
df1.head(3)
df2.head(3)
df3.head(3)

In [84]:
files = glob.glob("/Users/ajit/Downloads/NSE Data/EOD Options Snapshot - Nifty/nifty_daily_options_snapshot_2021*.csv")

In [103]:
df = pd.concat([pd.read_csv(file) for file in files], ignore_index = True)

In [104]:
df.shape

(178388, 14)

In [105]:
df.head()

Date Symbol     ExpiryDt  StrikePrice OptionTp    Open     High  \
0  05-MAY-2021  NIFTY  06-May-2021      12800.0       CE  1778.4  1819.20   
1  05-MAY-2021  NIFTY  06-May-2021      12850.0       CE     0.0     0.00   
2  05-MAY-2021  NIFTY  06-May-2021      12900.0       CE  1733.3  1733.30   
3  05-MAY-2021  NIFTY  06-May-2021      12950.0       CE  1630.2  1662.35   
4  05-MAY-2021  NIFTY  06-May-2021      13000.0       CE  1560.0  1640.00   

       Low    Close  SettlePrice  Contracts  ValueInLakhs    OI  OIChng  
0  1778.40  1819.20      1819.20          4         43.77   150       0  
1     0.00  1606.65      1769.15          0          0.00     0       0  
2  1731.80  1731.80      1731.80          2         21.94  1500       0  
3  1621.00  1662.35      1662.35          7         76.53  1500       0  
4  1556.25  1637.95      1637.95         26        284.22  1425    -225

In [117]:
df[(df.ExpiryDt == '24-Jun-2021') & (df.StrikePrice == 15000)]

Date Symbol     ExpiryDt  StrikePrice OptionTp    Open    High  \
1122    05-MAY-2021  NIFTY  24-Jun-2021      15000.0       CE  294.95  318.05   
1236    05-MAY-2021  NIFTY  24-Jun-2021      15000.0       PE  630.70  669.10   
3094    04-MAY-2021  NIFTY  24-Jun-2021      15000.0       CE  361.55  363.35   
3208    04-MAY-2021  NIFTY  24-Jun-2021      15000.0       PE  595.60  706.00   
5060    06-MAY-2021  NIFTY  24-Jun-2021      15000.0       CE  321.00  352.60   
...             ...    ...          ...          ...      ...     ...     ...   
173871  29-APR-2021  NIFTY  24-Jun-2021      15000.0       PE  472.00  550.85   
175588  30-APR-2021  NIFTY  24-Jun-2021      15000.0       CE  395.95  447.00   
175702  30-APR-2021  NIFTY  24-Jun-2021      15000.0       PE  600.00  654.15   
177544  03-MAY-2021  NIFTY  24-Jun-2021      15000.0       CE  280.25  366.75   
177658  03-MAY-2021  NIFTY  24-Jun-2021      15000.0       PE  698.85  769.40   

           Low   Close  SettlePrice  Contracts  ValueInLakhs      OI  OIChng  
1122    265.25  306.35       306.35       3662      42002.79  612300    -300  
1236    581.00  586.25       586.25       1182      13846.29  300900    5175  
3094    265.45  272.70       272.70       5265      60450.66  612600   71700  
3208    560.80  679.75       679.75       1517      17768.95  295725  -13350  
5060    296.05  340.35       340.35       4676      53714.52  666675   54375  
...        ...     ...          ...        ...           ...     ...     ...  
173871  440.55  525.60       525.60       3128      36364.58  336675   47250  
175588  348.05  353.25       353.25       3838      44306.30  511125   66900  
175702  534.85  627.95       627.95       1589      18565.72  326700   -9975  
177544  270.55  341.70       341.70       5481      62928.85  540900   29775  
177658  588.70  616.30       616.30       1750      20565.17  309075  -17625  

[168 rows x 14 columns]